# 12 Convolution Layer

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

    LABEL
        0 -> NORMAL (3,319)
        1 -> PNEUMOTHORAX (2,084)
        2 -> PNEUMONIA (4,277)

# hyper parameter

In [2]:
training_epochs = 500
batch_size = 20
image_resize = 1024
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

# label, test_label

In [3]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


# image, test_image

In [5]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

with tf.device('/gpu:0') :
    for i in range(len(image_list)) :
        image_list[i] = image_dir + image_list[i]
        image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))
#         image_array.append(np.array(PI.open(image_list[i]).convert('L')))

    for j in range(len(test_image_list)) :
        test_image_list[j] = test_image_dir + test_image_list[j]
        test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))
#         test_image_array.append(np.array(PI.open(test_image_list[j]).convert('L')))

    image_reshped = np.reshape(image_array, [-1, image_width*image_height])
    test_image_reshaped = np.reshape(test_image_array, [-1, image_width*image_height])

    image_array = np.array(np.reshape(image_reshped, [-1, image_width, image_height, 1]), dtype=np.float32)
    test_image_array = np.array(np.reshape(test_image_reshaped, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

MemoryError: 

# feeding value

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

is_training = tf.placeholder(dtype=tf.bool)

# learning layers

In [ ]:
def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

with tf.device('/gpu:0') :
    layer0_1 = tf.layers.conv2d(x, 8, [3,3], padding='SAME')
    layer0_2 = tf.layers.conv2d(layer0_1, 128, [3,3], padding='SAME')
    layer1 = learning_layer(input_data=layer0_2, size=256, dropout=0.7, is_training=is_training)
#     layer0_3 = tf.layers.conv2d(layer0_2, 256, [3,3], padding='SAME')
#     layer1 = learning_layer(input_data=layer0_2, size=256, drop_out=0.7, is_training=is_training, name_conv2d="layer1_conv2d", 
#                    name_maxpooling="layer1_maxpooling", name_dropout="layer1_dropout")
    layer1_1 = tf.layers.conv2d(layer1, 128, [3,3], padding='SAME')
    layer1_2 = tf.layers.conv2d(layer1_1, 256, [3,3], padding='SAME')
    layer2 = learning_layer(input_data=layer1_2, size=512, dropout=0.7, is_training=is_training)
    layer2_1 = tf.layers.conv2d(layer2, 256, [3,3], padding='SAME')
    layer2_2 = tf.layers.conv2d(layer2_1, 128, [3,3], padding='SAME')
#     layer2_3 = tf.layers.conv2d(layer2_2, 64, [3,3], padding='SAME')
    layer3 = learning_layer(input_data=layer2_2, size=64, dropout=0.7, is_training=is_training)
#     layer3 = learning_layer(input_data=layer2_2, size=64, drop_out=0.7, is_training=is_training, name_conv2d="layer3_conv2d", 
#                    name_maxpooling="layer3_maxpooling", name_dropout="layer3_dropout")
    layer3_1 = tf.layers.conv2d(layer3, 128, [3,3], padding='SAME')
    layer3_2 = tf.layers.conv2d(layer3_1, 256, [3,3], padding='SAME')
    layer4 = learning_layer(input_data=layer3_2, size=64, dropout=0.7, is_training=is_training)

In [ ]:
fully_layer = tf.contrib.layers.flatten(layer4)
fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu, name='fully_connected_layer')
fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')

fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu, name='fully_connected_layer2')
fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')

In [ ]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
    
with tf.device('/gpu:0') :
    with tf.name_scope('train') :
            train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [ ]:
print(label_output)
print(label_input)

In [ ]:
saver = tf.train.Saver()

tf.summary.scalar('scalar_loss', loss)

image_shaped_input = tf.reshape(x, [-1, image_resize, image_resize, 1])
tf.summary.image('input', image_shaped_input)

In [ ]:
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
#     f = open(log_dir+log_file, 'w')
    
#     saver = tf.train.import_meta_graph('meta_data_name')
#     saver.restore(sess, tf.train.latest_checkpoint('saved'))
    
    sess.run(tf.global_variables_initializer())
    
#     for var in tf.trainable_variables() :
#         tf.summary.histogram(var.name, var)

#     merged_summary = tf.summary.merge_all()  
#     train_writer = tf.summary.FileWriter(merge_dir, sess.graph)
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    
#     sensitivity_acc = 0
#     specificity_acc = 0
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_})    

#             _, _loss, _merge = sess.run([train, loss, merged_summary], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]],
#                                                                                   is_training: True, learning_rate: initial_rate*rate_})
            
#             saver.save(sess, save_dir, global_step=total_batch, write_meta_graph=False)
            
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
#     print("       SENSITIVITY ACC : ", '{:.5f}'.format(sensitivity_acc / 200))
#     print("       SPECIFICITY ACC : ", '{:.5f}'.format(specificity_acc / 100))
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
#     for cnt in range(len(correct_list)) :
#         data = correct_list[cnt]
#         f.write(str(data)+"\n")
#         print(data)
    
#     tf.train.write_graph(sess.graph, './', pb_, as_text=False)
#     tf.train.write_graph(sess.graph, './', txt_, as_text=True)
#     train_writer.add_graph(sess.graph)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
#     for cnt_incorrect in range(len(incorrect_list)) :
#         print(incorrect_list[cnt_incorrect])
    
#     f.close()
    
    coord.request_stop()
    coord.join(thread)